In [1]:
# STEP 2:

"""
Target:
The goal in this step is to add data augmentations, and to reduce to <8k params.


Result:
Random rotation was added.
Small fixes to StepLR params were made

num params: 7.5k
train accuracy: 99.06
test accuracy: 99.33


Analysis:
Training accuracy reduced as expected.
Now test accuracy > train accuracy

Accuracy is not improving though.
Remaining stable after 10th epoch

"""




In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.optim.lr_scheduler import StepLR, OneCycleLR


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    train_loss = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('\n#Train set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(
        train_loss,
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('#Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')

torch.manual_seed(1)
batch_size = 128
lr=0.01

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7, 7), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

print(f'Num images in train: {len(train_loader.dataset)}')

device: cpu
Num images in train: 60000


In [ ]:
# From step 1 (change gamma to 0.333)

class Net_14(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=0)   #input - 28, Output 26, RF
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, 3, padding=0)   #input - 26, Output 24, RF
        self.bn2 = nn.BatchNorm2d(10)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(10, 24, 3, padding=0)   #input - 24, Output 22, RF
        self.bn3 = nn.BatchNorm2d(24)

        self.conv4 = nn.Conv2d(24, 32, 3, padding=0) #input - 11, Output 9, RF
        self.bn4 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 10, 3, padding=0) #input - 9, Output 7, RF

        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.conv6 = nn.Conv2d(10, 10, 1)            #input - 7, Output 7, RF

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)
        self.dropout5 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)



        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_14
lr = 0.1
step_size = 5
gamma = 1./3

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
#Train set: Average loss: 0.0002, Accuracy: 99.08%
#Test set: Average loss: 0.0209, Accuracy: 9932/10000 (99.32%)

#epoch: 20
#Train set: Average loss: 0.0002, Accuracy: 99.31%
#Test set: Average loss: 0.0181, Accuracy: 9935/10000 (99.35%)





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
           Dropout-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
           Dropout-6           [-1, 10, 24, 24]               0
         MaxPool2d-7           [-1, 10, 12, 12]               0
            Conv2d-8           [-1, 24, 10, 10]           2,184
       BatchNorm2d-9           [-1, 24, 10, 10]              48
          Dropout-10           [-1, 24, 10, 10]               0
           Conv2d-11             [-1, 32, 8, 8]           6,944
      BatchNorm2d-12             [-1, 32, 8, 8]         

<ipython-input-14-4ac50ce3d6ae>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# rebalance the params.

class Net_15(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0)   #input - 28, Output 26, RF
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=0)   #input - 26, Output 24, RF
        self.bn2 = nn.BatchNorm2d(16)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(16, 24, 3, padding=0)   #input - 24, Output 22, RF
        self.bn3 = nn.BatchNorm2d(24)

        self.conv4 = nn.Conv2d(24, 24, 3, padding=0) #input - 11, Output 9, RF
        self.bn4 = nn.BatchNorm2d(24)

        self.conv5 = nn.Conv2d(24, 16, 3, padding=0) #input - 9, Output 7, RF
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.conv6 = nn.Conv2d(16, 10, 1)            #input - 7, Output 7, RF

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)
        self.dropout5 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_15
lr = 0.1
step_size = 5
gamma = 1./3

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#Epoch 10
#Train set: Average loss: 0.0002, Accuracy: 99.15%
#Test set: Average loss: 0.0204, Accuracy: 9937/10000 (99.37%)

#Epoch 20:
#Train set: Average loss: 0.0002, Accuracy: 99.36%
#Test set: Average loss: 0.0167, Accuracy: 9945/10000 (99.45%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 24, 10, 10]           3,480
       BatchNorm2d-9           [-1, 24, 10, 10]              48
          Dropout-10           [-1, 24, 10, 10]               0
           Conv2d-11             [-1, 24, 8, 8]           5,208
      BatchNorm2d-12             [-1, 24, 8, 8]         

<ipython-input-15-02105f051fc3>:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# reduce the params.

class Net_16(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 12, 3, padding=0)   #input - 28, Output 26, RF
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, 12, 3, padding=0)   #input - 26, Output 24, RF
        self.bn2 = nn.BatchNorm2d(12)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(12, 16, 3, padding=0)   #input - 24, Output 22, RF
        self.bn3 = nn.BatchNorm2d(16)

        self.conv4 = nn.Conv2d(16, 16, 3, padding=0) #input - 11, Output 9, RF
        self.bn4 = nn.BatchNorm2d(16)

        self.conv5 = nn.Conv2d(16, 12, 3, padding=0) #input - 9, Output 7, RF

        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.conv6 = nn.Conv2d(12, 10, 1)            #input - 7, Output 7, RF

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)
        self.dropout5 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)



        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_16
lr = 0.1
step_size = 5
gamma = 1./3

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#Epoch 10
#Train set: Average loss: 0.0002, Accuracy: 99.05%
#Test set: Average loss: 0.0232, Accuracy: 9934/10000 (99.34%)

#Epoch 20:
#Train set: Average loss: 0.0002, Accuracy: 99.12%
#Test set: Average loss: 0.0213, Accuracy: 9936/10000 (99.36%)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 26, 26]             120
       BatchNorm2d-2           [-1, 12, 26, 26]              24
           Dropout-3           [-1, 12, 26, 26]               0
            Conv2d-4           [-1, 12, 24, 24]           1,308
       BatchNorm2d-5           [-1, 12, 24, 24]              24
           Dropout-6           [-1, 12, 24, 24]               0
         MaxPool2d-7           [-1, 12, 12, 12]               0
            Conv2d-8           [-1, 16, 10, 10]           1,744
       BatchNorm2d-9           [-1, 16, 10, 10]              32
          Dropout-10           [-1, 16, 10, 10]               0
           Conv2d-11             [-1, 16, 8, 8]           2,320
      BatchNorm2d-12             [-1, 16, 8, 8]         

<ipython-input-16-20853ad8100a>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:


optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

for epoch in range(1, 26):
    print(f'#epoch: {epoch}')
    print(f'#lr: {optimizer.param_groups[0]["lr"]}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()


#epoch: 1
#lr: 0.1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-16-20853ad8100a>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.15269608795642853 batch_id=468: 100%|██████████| 469/469 [01:22<00:00,  5.71it/s]



#Train set: Average loss: 0.0027, Accuracy: 88.57%
#Test set: Average loss: 0.0961, Accuracy: 9673/10000 (96.73%)

#epoch: 2
#lr: 0.1


loss=0.037718188017606735 batch_id=468: 100%|██████████| 469/469 [01:17<00:00,  6.07it/s]



#Train set: Average loss: 0.0007, Accuracy: 97.32%
#Test set: Average loss: 0.0577, Accuracy: 9804/10000 (98.04%)

#epoch: 3
#lr: 0.1


...


#Train set: Average loss: 0.0002, Accuracy: 99.05%
#Test set: Average loss: 0.0198, Accuracy: 9937/10000 (99.37%)

#epoch: 24
#lr: 0.0012345679012345677


loss=0.037076275795698166 batch_id=468: 100%|██████████| 469/469 [01:16<00:00,  6.11it/s]



#Train set: Average loss: 0.0002, Accuracy: 99.07%
#Test set: Average loss: 0.0196, Accuracy: 9941/10000 (99.41%)

#epoch: 25
#lr: 0.0012345679012345677


loss=0.042446281760931015 batch_id=468: 100%|██████████| 469/469 [01:16<00:00,  6.12it/s]



#Train set: Average loss: 0.0002, Accuracy: 99.10%
#Test set: Average loss: 0.0195, Accuracy: 9933/10000 (99.33%)

